In [1]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.callbacks import TensorBoard, ModelCheckpoint, CSVLogger
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, Dense
from data_utils import DataSet

import numpy as np
import os.path
import time

/Users/vincentchow/anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_cnn_model():
    base_model = InceptionV3(weights='imagenet', include_top=True)
    
    print base_model.summary()
#     print base_model.get_layer('global_max_pooling2d_1').output
    outputs = base_model.get_layer('avg_pool').output

    cnn_model = Model(inputs=base_model.input, outputs=outputs)

    return cnn_model


In [3]:
cnn_model = load_cnn_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# Creating Sequences

## Looks to be ~ 12 seconds per video

### ~24 minutes for 120 videos, ~6.6 hrs for entire 1980 videos

In [4]:
# ---- RUN THE FOLLOWING CELLS TO GENERATE .NPY SEQ FOR ALL VIDEOS IN DATASET ----- # 
#      SEQUENCES THAT ALREADY EXIST WILL BE SKIPPED

seq_length = 16
dataset = DataSet(cnn_model)

In [6]:
dataset.data[-1]

['test', 'smash', 'p9_smash_s3']

In [7]:
tic = time.time()

for ind, sample in enumerate(dataset.data):
    
    path = os.path.join('data', 'sequences', sample[1], sample[2] + '-' + str(seq_length) + \
        '-features.npy')
    
    if os.path.isfile(path):
        print "Sequence: {} already exists".format(ind)
    else:
        print "Generating and saving sequence: {}".format(ind)
        sequence = dataset.extract_seq_features(sample)

print "Time Elapsed: {}".format(time.time() - tic)

Sequence: 0 already exists
Sequence: 1 already exists
Sequence: 2 already exists
Sequence: 3 already exists
Sequence: 4 already exists
Sequence: 5 already exists
Sequence: 6 already exists
Sequence: 7 already exists
Sequence: 8 already exists
Sequence: 9 already exists
Sequence: 10 already exists
Sequence: 11 already exists
Sequence: 12 already exists
Sequence: 13 already exists
Sequence: 14 already exists
Sequence: 15 already exists
Sequence: 16 already exists
Sequence: 17 already exists
Sequence: 18 already exists
Sequence: 19 already exists
Sequence: 20 already exists
Sequence: 21 already exists
Sequence: 22 already exists
Sequence: 23 already exists
Sequence: 24 already exists
Sequence: 25 already exists
Sequence: 26 already exists
Sequence: 27 already exists
Sequence: 28 already exists
Sequence: 29 already exists
Sequence: 30 already exists
Sequence: 31 already exists
Sequence: 32 already exists
Sequence: 33 already exists
Sequence: 34 already exists
Sequence: 35 already exists
Se

# Sanity Check on Generated Sequences

In [8]:
path = os.path.join('data', 'sequences', 'backhand', 'p36_backhand_s3-16-features.npy')

sequence = np.load(path)
sequence

array([[0.63280857, 0.33544415, 0.3047987 , ..., 0.27660683, 0.40139177,
        0.37127692],
       [0.66674316, 0.29698414, 0.24121535, ..., 0.29314587, 0.5353388 ,
        0.34506497],
       [0.7082304 , 0.35714126, 0.31769606, ..., 0.30259886, 0.37390557,
        0.29302064],
       ...,
       [0.65532845, 0.34075433, 0.2506685 , ..., 0.36052316, 0.5520053 ,
        0.42216247],
       [0.6798128 , 0.36810336, 0.2818408 , ..., 0.30228627, 0.5648415 ,
        0.3953814 ],
       [0.7253652 , 0.36406755, 0.24089412, ..., 0.3171352 , 0.52032834,
        0.38690823]], dtype=float32)